In [2]:
## Arxiv research paper
## Tools creation

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [3]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.invoke("what is the name of indian prime minister")

'Page: List of prime ministers of India\nSummary: The prime minister of India is the chief executive of the Government of India and chair of the Union Council of Ministers. Although the president of India is the constitutional, nominal, and ceremonial '

In [4]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [5]:
tools=[wiki,arxiv]


In [6]:
## Making custom tools
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
load_dotenv()



embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv("HUGGINGFACEHUB_API_TOKEN")

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\ritik\anaconda3\envs\search\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=250)
chunks=splitter.split_documents(docs)
vectordb=FAISS.from_documents(chunks,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000206E34614B0>, search_kwargs={})

## CONVERTING THE TOOL INTO A RETRIEVER

In [17]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,name="Lagmsith-search",description="Search any information about Langsmith")
retriever_tool.description

'Search any information about Langsmith'

In [19]:
tools=[wiki,arxiv,retriever_tool]

In [22]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\ritik\\anaconda3\\envs\\search\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Lagmsith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000206DE145E10>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000206E34

In [32]:
## Run all this tools with Agents and LLM models

## Tools
from langchain_groq import ChatGroq
api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=api_key , model_name="Llama3-8b-8192")

In [39]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [38]:
## Agent
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [ ]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [44]:
agent_executor.invoke({"input":"what is attention is all you need research paper"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Attention is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,
Invoking: `wikipedia` with `{'query': 'Transformer (machine learning)'}`


Page: Transformer (deep learning architecture)
Summary: In deep learning, transformer is an architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and each token is converted i
Invoking: `arxiv` with `{'query': 'Transformer (machine learning)'}`


Published: 2025-02-03
Title: Aspects of Artificial Intelligence: Transforming Machine Learning Systems Naturally
Authors: Xiuzhan Guo
Summary: In this paper, we study the machine learning elements which we are interested
in together as a mac

{'input': 'what is attention is all you need research paper',
 'output': 'The paper "Aspects of Artificial Intelligence: Transforming Machine Learning Systems Naturally" seems to be a recent publication. However, it appears to be unrelated to the original topic of "Attention is All You Need" and the Transformer architecture.\n\nSince we have already discussed the Transformer architecture and the original research paper, I\'ll provide a summary:\n\nThe Transformer architecture, introduced in the "Attention Is All You Need" paper, revolutionized the field of natural language processing by providing a new way to process sequential data without relying on recurrence or convolution. The architecture is based on self-attention mechanisms, which allow the model to attend to different parts of the input sequence simultaneously.\n\nAs the conversation has already covered the Transformer architecture, I\'ll provide a direct response:\n\nThe Transformer architecture has been widely adopted in man

In [45]:
agent_executor.invoke({"input":"what is the paper 1706.03762 all about"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'paper 1706.03762'}`


Published: 2020-02-12
Title: GLU Variants Improve Transformer
Authors: Noam Shazeer
Summary: Gated Linear Units (arXiv:1612.08083) consist of the component-wise product
of two linear projections, one of which is first passed through a sigmoid
functioBased on the result yielded by the tool call, it appears that the paper "GLU Variants Improve Transformer" (arXiv:1706.03762) is about improving the Transformer model using Gated Linear Units (GLU).

> Finished chain.


{'input': 'what is the paper 1706.03762 all about',
 'output': 'Based on the result yielded by the tool call, it appears that the paper "GLU Variants Improve Transformer" (arXiv:1706.03762) is about improving the Transformer model using Gated Linear Units (GLU).'}